## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
load_data = "../Weather_Database/WeatherPy_database.csv"
city_data_df = pd.read_csv(load_data)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nanakuli,US,21.3906,-158.1547,87.75,51,20,12.66,few clouds
1,1,Torbay,CA,47.6666,-52.7314,74.17,49,40,20.71,scattered clouds
2,2,Hobart,AU,-42.8794,147.3294,39.06,91,75,6.91,broken clouds
3,3,Norman Wells,CA,65.2820,-126.8329,55.44,82,100,11.50,overcast clouds
4,4,Lagoa,PT,39.0500,-27.9833,68.58,93,67,15.37,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Torbay,CA,47.6666,-52.7314,74.17,49,40,20.71,scattered clouds
3,3,Norman Wells,CA,65.2820,-126.8329,55.44,82,100,11.50,overcast clouds
4,4,Lagoa,PT,39.0500,-27.9833,68.58,93,67,15.37,broken clouds
5,5,Benito Juarez,MX,19.3850,-99.1650,78.62,27,40,9.22,scattered clouds
7,7,Murdochville,CA,48.9583,-65.5006,59.88,64,76,10.94,broken clouds
8,8,Atuona,PF,-9.8000,-139.0333,78.13,73,26,19.04,scattered clouds
9,9,Makokou,GA,0.5738,12.8642,70.27,86,43,4.59,scattered clouds
10,10,Bredasdorp,ZA,-34.5322,20.0403,59.56,72,100,18.75,overcast clouds
14,14,Ucluelet,CA,48.9329,-125.5528,64.71,82,23,5.01,few clouds
17,17,Rikitea,PF,-23.1203,-134.9692,72.79,79,89,9.26,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
no_null_preferred_cities_df = preferred_cities_df.dropna()
no_null_preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = no_null_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Torbay,CA,74.17,scattered clouds,47.6666,-52.7314,
3,Norman Wells,CA,55.44,overcast clouds,65.2820,-126.8329,
4,Lagoa,PT,68.58,broken clouds,39.0500,-27.9833,
5,Benito Juarez,MX,78.62,scattered clouds,19.3850,-99.1650,
7,Murdochville,CA,59.88,broken clouds,48.9583,-65.5006,
8,Atuona,PF,78.13,scattered clouds,-9.8000,-139.0333,
9,Makokou,GA,70.27,scattered clouds,0.5738,12.8642,
10,Bredasdorp,ZA,59.56,overcast clouds,-34.5322,20.0403,
14,Ucluelet,CA,64.71,few clouds,48.9329,-125.5528,
17,Rikitea,PF,72.79,light rain,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
no_missing_hotels_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
no_missing_hotels_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Torbay,CA,74.17,scattered clouds,47.6666,-52.7314,Gallows Cove
3,Norman Wells,CA,55.44,overcast clouds,65.2820,-126.8329,Heritage Hotel
4,Lagoa,PT,68.58,broken clouds,39.0500,-27.9833,Casa Das Faias
5,Benito Juarez,MX,78.62,scattered clouds,19.3850,-99.1650,Hotel Beverly
7,Murdochville,CA,59.88,broken clouds,48.9583,-65.5006,Les Suites économique Murdochville
8,Atuona,PF,78.13,scattered clouds,-9.8000,-139.0333,Villa Enata
9,Makokou,GA,70.27,scattered clouds,0.5738,12.8642,Hôtel
10,Bredasdorp,ZA,59.56,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
14,Ucluelet,CA,64.71,few clouds,48.9329,-125.5528,Black Rock Oceanfront Resort
17,Rikitea,PF,72.79,light rain,-23.1203,-134.9692,People ThankYou


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
no_missing_hotels_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in no_missing_hotels_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = no_missing_hotels_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))